## Homework 1

### CS 490A, UMass CICS, Fall 2022
### Due: September 21st at 11:59pm. Please see detailed submission instructions below.  70 points total.

##### How to complete this homework:

- Use Python version 3.  We strongly suggest installing Python from the [Anaconda Individual Edition](https://docs.anaconda.com/anaconda/) software package.

- Add code or textual answers where prompted.

- This assignment is designed so that you can run all cells in several minutes of computation time. If it is taking longer than that, you probably have a mistake in your code.

##### How to submit this homework:
- Write all the answers in this notebook. Once you are finished, (1) Generate a PDF via (File -> Download As -> PDF)  (2) Upload your PDF file to Gradescope. (3) Upload your notebook to Gradescope.

- **Important:** Check your PDF before you upload it to Gradescope to make sure it exported correctly. If the notebook gets confused about your syntax, it will sometimes terminate the PDF creation routine early. You are responsible for checking for these errors. If your whole PDF does not print, try running on the commandline `jupyter nbconvert --to pdf hw1.ipynb` to identify and fix any syntax errors that might be causing problems.

- **Important:** When creating your final version of the PDF, please do a fresh restart and execute every cell in order. Then you'll be sure it's actually right. One convenient way to do this is by clicking `Cell -> Run All` in the notebook menu.

- If you are having trouble with PDF export, you can always paste screenshots into a word processor then turn that into PDF.


##### Academic honesty:
- Like always, check the course's collaboration / academic honesty policy that is available within the course syllabus (posted on Moodle). All of the content you submit for this homework, both code and text, must be your own independent work. Do not share code or written materials, and list any classmates that you had high-level discussions with. We will check your submissions for plagiarism, and we will follow up collaboration policy violations with the university’s Academic Honesty Policy and Procedures.

## Collaboration Declarations:
List here the name of any classmates that you had high-level discussions with about the homework.

**OPTIONAL: Your declarations here**

In [2]:
# Make sure to run this cell, it imports all the packages we will need for this assignment
import re
import math
import nltk
from collections import Counter
from tqdm.notebook import tqdm # For progress bar in Part Two
from nltk.tokenize import word_tokenize # For tokenization in Part Two
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


False

## Part One: Text Processing

### Regular Expressions

Consider the following passage:
$$
\text{"It is very seldom," the young man said at last, "that dragons ask to do men favors."}
$$

and the following regular expression:
$$
\text{".+"}
$$

**Question 1.1 (5 pts)** Describe what this regular expression will match on.

**Your answer here**  
The regular expression of ".+" will match with one or more occurrences of any char except the new line character.

**Question 1.2 (5 pts)** List each of the matching substrings from the passage for this regular expression. 

**Your answer here**   
The matching substring from the passage will be every possible substring of the text and it can not be empty. 
Passage: "It is very seldom," the young man said at last, "that dragons ask to do men favors."
So every Non-empty substring matches regular expression of ".+"
For example:  
"I"  
"It"  
"It "  
"It i"  
"It is"  
"It is "  
This will go on till the last substring that will be equal to the complete passage.
Also, other substrings such as "is", "that" and every possible Non-empty substring will match.

Now, let's try using regular expressions in Python using the [`re`](https://docs.python.org/3/library/re.html) built-in package. The following method `get_matches` prints each match for a regular expression `regex` and input string `text` using [`re.findall`](https://docs.python.org/3/library/re.html#re.findall). 

In [5]:
def get_matches(regex, text):
    matches = re.findall(regex, text)
    print("Matches:")
    for i, match in enumerate(matches):
        print("{}. {}".format(i+1, match))

The following cell runs `get_matches` on our working passage and regular expression.

In [13]:
passage = '"It is very seldom," the young man said at last, "that dragons ask to do men favors."'
pattern = '".+"'

get_matches(pattern, passage)

Matches:
1. "It is very seldom," the young man said at last, "that dragons ask to do men favors."


**Question 1.3 (5 pts)** Describe how the results from the previous code cell differ from your answer in **(b)**. Why are they different?

*HINT: Take a look at this [page](https://docs.python.org/3/howto/regex.html#greedy-versus-non-greedy).*

**Your answer here**  
The result did not give all the possible matches for the regular expression ".+".  
They are different since the re.findall in get_matches would return all non-overlapping matches of pattern in string, as a list of strings. But in my answer in (b) there are lots of overlapping matches such as "It" and "It is".

Now, let's try using a *non-greedy* version of the `+` operator. 

In [14]:
passage = '"It is very seldom," the young man said at last, "that dragons ask to do men favors."'
pattern = '".+?"'

get_matches(pattern, passage)

Matches:
1. "It is very seldom,"
2. "that dragons ask to do men favors."


**Question 1.4 (5 points)** Describe how these results differ from **1.2** and **1.3**. Why do these results differ?

**Your answer here**  
For pattern ".+", it is greedy and consumes as many characters as it can. However, for pattern ".+?" is reluctant and consumes as few characters as it can.

### Text Processing with Python
Now, let's use regular expressions to extract and process text using Python. We'll be working with the following file `Avatar_2009_script.txt` which contains a script of James Cameron's 2009 film Avatar. The sequel, Avatar 2, is expected to release this year, but it's been quite a few years since the original film came out, so let's try using text processing to learn a bit more about the original movie.

We'll focus on learning about the characters of this film and how often they have speaking lines within the script. We can use the *structure* of the text file to do so.

In the following excerpt, the character speaking lines are *italicized* and the characters speaking these lines is **bolded**. For visualization purposes spaces are being represented as "·"s.

<pre>
··········Some of the other scientists clap Jake on the shoulders in
··········congratulation.
··········
······························<b>MAX</b>
····················<i>That's awesome, Jake.</i>
··········
··········NORM chomps his bacon, fuming.
··········
······························<u>·</u><b>GRACE</b>
·························(to Jake, getting serious)
····················<i>For reasons I cannot fathom, the</i>
····················<i>Omaticaya have chosen you. God help us</i>
····················<i>all.</i>
·································································CUT TO:
··········
··········INT. OPS CENTER - MORNING
</pre>

*Note that this structure isn't perfect. There are inconsistencies, one is <u>underlined</u> above.*

**Question 1.5 (5 points)** Examine the structure of the script and describe how *dialogue* is organized within the script. How are the lines of spoken dialogue organized? How are the character names assigned for each line of dialogue organized?

**Your answer here**  
For dialogue, it is first with name of who is gonna speak which is in bold and then probablily also a () with a little explaination in it about who is the character speaking to and under what situation.   
The character names is after 30 space ; () with explaination is after 25 space ; dialogue is after 20 space

**Question 1.6 (5 points)** Modify the regular expression `pattern` in the following code cell so that it will only match on the character name that corresponds to each instance of dialogue. We'll accumulate these matches into a [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) `character_counts`.

For example, the following excerpt will result in a single match `"MAX"`.

<pre>
······························MAX
····················That's awesome, Jake.
</pre>

*Note that since this structure isn't perfect, make a best effort at extracting character names.*

In [186]:
pattern = "^[ ]{30}([A-Z]{1,10})"  # Modify this pattern "\w+S+"

character_counts = Counter()
with open("Avatar_2009_script.txt", encoding="utf-8") as reader:
    for line in reader:
        matches = re.findall(pattern, line)
        if matches:
            print(matches)
            character_counts.update(matches)

['JAKE']
['JAKE']
['JAKE']
['JAKE']
['JAKE']
['JAKE']
['JAKE']
['JAKE']
['AGENT']
['AGENT']
['JAKE']
['JAKE']
['JAKE']
['AGENT']
['JAKE']
['JAKE']
['JAKE']
['MED']
['MED']
['CREW']
['CREW']
['CREW']
['CREW']
['WAINFLEET']
['FIKE']
['WAINFLEET']
['JAKE']
['WAINFLEET']
['MAN']
['QUARITCH']
['QUARITCH']
['QUARITCH']
['JAKE']
['NORM']
['NORM']
['NORM']
['JAKE']
['NORM']
['MAX']
['NORM']
['MAX']
['JAKE']
['NORM']
['JAKE']
['NORM']
['JAKE']
['MAX']
['NORM']
['MAX']
['GRACE']
['MAX']
['GRACE']
['NORM']
['GRACE']
['NORM']
['MAX']
['GRACE']
['JAKE']
['GRACE']
['GRACE']
['JAKE']
['GRACE']
['MAX']
['MAX']
['GRACE']
['GRACE']
['SELFRIDGE']
['SELFRIDGE']
['GRACE']
['SELFRIDGE']
['SELFRIDGE']
['GRACE']
['SELFRIDGE']
['GRACE']
['NORM']
['JAKE']
['GRACE']
['JAKE']
['GRACE']
['JAKE']
['MAX']
['MAX']
['TECH']
['MAX']
['MED']
['MAX']
['GRACE']
['GRACE']
['GRACE']
['NORM']
['NORM']
['QUARITCH']
['TROOPERS']
['QUARITCH']
['JAKE']
['GRACE']
['GRACE']
['JAKE']
['JAKE']
['TRUDY']
['QUARITCH']
['QUARITCH']
['J

**Question 1.7 (5 points)** Now let's use `character_counts` so we can examine the dialogue counts for characters in Avatar.

Write code in the following cell, so that it prints a line for each character containing the character's name and their dialogue count. These lines must be *sorted* in descending order by dialogue counts.

```
[Character] [Highest Dialogue Count]
[Character] [Second Highest Dialogue Count]
.
.
.
[Character] [Lowest Dialogue Count]
```

In [137]:
# Write code here
temp = character_counts.most_common()
for key in temp:
    print('[' + key[0] + '] [' + str(key[1]) +']')

[JAKE] [210]
[GRACE] [113]
[NEYTIRI] [63]
[QUARITCH] [54]
[NORM] [35]
[SELFRIDGE] [33]
[TRUDY] [26]
[MO] [25]
[TSU] [21]
[MAX] [19]
[EYTUKAN] [12]
[WAINFLEET] [10]
[CREW] [4]
[AGENT] [3]
[MED] [3]
[TROOPER] [2]
[PILOT] [2]
[FIKE] [1]
[MAN] [1]
[TECH] [1]
[TROOPERS] [1]
[MECHANIC] [1]
[OPERATOR] [1]
[SUPERVISOR] [1]
[PILOTS] [1]
[GUARD] [1]
[CROWD] [1]


**Question 1.8 (5 points)** What does this output tell us about Avatar? If you haven't seen the film or don't remember it, feel free to refer to [Wikipedia](https://en.wikipedia.org/wiki/Avatar_(2009_film)) and [IMDB](https://www.imdb.com/title/tt0499549/). Are the characters with the most spoken dialogue who you expect? Are there characters that have surprisingly little dialogue? Explain your reasoning.

**Your answer here**  
This output tells me that 'Jake' is the male protagonist, and 'Grac' is the female protagonist. And other charaters such as 'Neytiri' and 'Quaritch' and 'Norm' and 'Selfridge' and others are also characters in Avatar.  
Jake speak most dialogue and this is also what I am expecting.  
I am sprsingling see little dialogue for guard and crowd since they seem to speak may be more than once in Avatar.

## Part Two: Language Modeling
In this part we will implement several n-gram language models. We will train and evaluate these language models using a [collection of movie summaries](http://www.cs.cmu.edu/~ark/personas/).

### Setup: Building working datasets

In [138]:
def extract_tokens(text):
    tokens = ["<bos>", "<bos>"]
    clean_text = " ".join(text.strip().lower().split())
    tokens.extend(word_tokenize(clean_text))
    tokens.append("<eos>")
    return tokens

In [139]:
# Decide train and test set size (in number of plot summaries)
train_size = 20000
test_size = 5000

train_set = []
test_set = []

with open('plot_summaries.txt', encoding='utf-8') as reader:
    # Extract tokens for training set
    print("Building training set")
    for i in tqdm(range(train_size)):
        line = reader.readline()
        summary_text = line.split('\t')[1]
        train_set.extend(extract_tokens(summary_text))
    # Extract tokens for test set
    print("Building test set")
    for i in tqdm(range(test_size)):
        line = reader.readline()
        summary_text = line.split('\t')[1]
        test_set.extend(extract_tokens(summary_text))

# Vocabulary information
word_counts = Counter(train_set)
vocab = sorted(word_counts.keys())
vocab_size = len(vocab)

print("Vocabulary size: {} word types".format(vocab_size))
print("Example text: {}".format(train_set[:20]))

Building training set


  0%|          | 0/20000 [00:00<?, ?it/s]

Building test set


  0%|          | 0/5000 [00:00<?, ?it/s]

Vocabulary size: 116191 word types
Example text: ['<bos>', '<bos>', 'shlykov', ',', 'a', 'hard-working', 'taxi', 'driver', 'and', 'lyosha', ',', 'a', 'saxophonist', ',', 'develop', 'a', 'bizarre', 'love-hate', 'relationship', ',']


### N-gram Language Models

First, let's build code to build n-gram models. To train these models, we will use the n-gram (and n-1 gram) counts from our training set to compute the relative frequencies of n-grams. To ensure that no word in the vocabulary has $0$ probability, we'll use additive smoothing. More formally, we'll compute the probability of the next word as follows:

$$\Pr(w_n \mid w_1 \dots w_{n-1}) = 
\frac{C(w_1 \dots w_n) + \alpha}{C(w_1 \dots w_{n-1}) + \alpha \cdot |V|}$$

where $C$ is the count for a given n-gram (in the training set) and $|V|$ is the size of the working vocabulary.

**Question 2.1 (10 points)** The following code cell contains a partial implementation of a class for n-gram models, `NGramModel`. Implement its `get_final_word_prob` method. It computes the probability for the final word of the input text, `tokens` which is a list of string tokens.

- *Hints:* 
- Make sure to use the equation listed above
- `self.n_counts` contains the observed counts for `self.n` grams
- `self.n_1_counts` contains the observed counts for `self.n-1` grams
- Use the [`join`](https://docs.python.org/3/library/stdtypes.html#str.join) to create a string from a list of strings
- `self.vocab` contains a list of the vocabulary
- Remember that input text can be larger than `self.n`

In [324]:
class NGramModel:
    # Initialize n-gram model
    def __init__(self, train_tokens, vocab, n, alpha=1e-3):
        self.n = n
        self.smoothing = alpha
        self.vocab = vocab
        
        # Gather n-gram counts
        self.n_counts = Counter()
        current_tokens = ["<bos>"] * (self.n - 1) # Padding for initial word
        for token in train_tokens:
            current_tokens.append(token) # Add next token
            n_gram = " ".join(current_tokens)
            self.n_counts[n_gram] += 1
            current_tokens.pop(0) # Remove first token
        
        #print(self.n_counts)
        
        # Gather n-1 gram counts
        self.n_1_counts = Counter()
        for n_gram, count in self.n_counts.items():
            n_1_gram = " ".join(n_gram.split()[:-1])
            self.n_1_counts[n_1_gram] += count
            
        #print(self.n_1_counts)
    
    # Return the probability for the final word of an n-gram.
    def get_final_word_prob(self, tokens):
        assert len(tokens) >= self.n, "ERROR: Input too short. Expected {}+ tokens, but received {}.".format(
            self.n, len(tokens))
        # WRITE YOUR CODE HERE
        lamda = self.smoothing
        tokenList = tokens.split();
        if len(tokenList[-1]) <2 :
            word = tokenList[-2]
            n_before = " ".join(tokenList[-2-self.n:-2])
        else:
            n_before = " ".join(tokenList[-1-self.n:-1])
            word = tokenList[-1]
        if self.n == 1:
            word = ''
        #print('aaa')
        #print(tokens)
        #print(word)
        #print(n_before)
        #print(self.n_counts[n_before])
        #print(self.n_1_counts[word])
        return (self.n_counts[n_before]+lamda)/(self.n_1_counts[word]+lamda*len(self.vocab))

### Perplexity

To evaluate the quality of our language models, we use an *intrinsic* evaluation metric called perplexity.
The perplexity (PP) on a test set is the inverse probability of the test set normalized by the number of words.
For a test set $W=w_1w_2\dots w_N$, perplexity is defined as follows:

$$PP(W)=\Pr(w_1w_2\dots w_N)^{-\frac{1}{N}} = \sqrt[N]{\frac{1}{\Pr(w_1w_2\dots w_N)}}$$

Using the chain rule we can reformulate perplexity as follows:
$$PP(W) = \sqrt[N]{\frac{1}{\prod_{i=1}^N\Pr(w_i \mid w_1 \dots w_{i-1})}}$$

Since we'll be multiplying quite a few probabilities (many of which are very small), we'll first compute values in the log domain to avoid numerical underflow. Then, we can compute our final perplexity score by exponentiating our log perplexity score. As a result, we'll use the following two equations:

$$\ln PP(W) = -\frac{1}{N}\sum_{i=1}^{N}\ln\Pr(w_i \mid w_1 \dots w_{i-1})$$

$$PP(W) = e^{\ln PP(W)}$$

**Question 2.2 (10 pts)** Implement the following method `get_perplexity` which takes an NGramModel `model` and test set `test_tokens` (list of string tokens) as input, and returns the perplexity score of `model` for `test_tokens`.

Hints:
- Your implementation will use `model.get_probability`
- Your implementation should use `math.log` and `math.exp`
- Remember to add padding (use `<bos>` tokens) for the first n-1 tokens in the test set
- You will need to compute the probability of each successive n-gram

In [325]:
def get_perplexity(model, test_tokens):
    # WRITE YOUR CODE HERE
    prob = 0.0
    count = 0
    i = 0
    str_temp = ''
    while i < len(test_tokens):
        if test_tokens[i] == '<bos>':
            i+=1
        elif test_tokens[i] == '<eos>':
            prob += math.log(model.get_final_word_prob(str_temp))
            #print(model.get_final_word_prob(str_temp))
            i+=1
            count+=1
            str_temp = ''
        else:
            str_temp += ' '+test_tokens[i]
            i+=1
    help_p = (-float(1)/count)*prob 
    return math.exp(help_p)

Now, let's train unigram, bigram, and trigram models using our training set `train_set` and measure the perplexity score of each model for our test set `test_set`.

In [326]:
unigram_model = NGramModel(train_set, vocab, n=1)

# This score should be around 1300
print('unigram test perplexity: {:.1f}\n'.format(get_perplexity(unigram_model, test_set)))

# This score should be around 600
bigram_model = NGramModel(train_set, vocab, n=2)
print('bigram test perplexity: {:.1f}\n'.format(get_perplexity(bigram_model, test_set)))

# This score should be around 3300-3400
trigram_model = NGramModel(train_set, vocab, n=3)
print('trigram test perplexity: {:1f}\n'.format(get_perplexity(trigram_model, test_set)))

unigram test perplexity: 1368.3

bigram test perplexity: 67.0

trigram test perplexity: 999.779008



**Question 2.3 (5 pts)** Why does the trigram model perform so poorly (i.e. have high perplexity)?

**Your answer here**  
Since our |v| = 116191, in this case, with lower n value such as 1,2,3, it is hard to predict the letter after n letter before with this high vocab which result high prelexity.

What if we instead measured the perplexity of our trained models for our training set `train_set`? 

In [307]:
print('unigram test perplexity: {:.1f}\n'.format(get_perplexity(unigram_model, train_set)))

bigram_model = NGramModel(train_set, vocab, n=2)
print('bigram test perplexity: {:.1f}\n'.format(get_perplexity(bigram_model, train_set)))

trigram_model = NGramModel(train_set, vocab, n=3)
print('trigram test perplexity: {:1f}\n'.format(get_perplexity(trigram_model, train_set)))

unigram test perplexity: 1127.9

bigram test perplexity: 18.9

trigram test perplexity: 28.570622



**Question 2.4 (5 points)** How do these perplexity scores and their orderings differ from those of **2.3**? Is this surprising? Explain your reasoning.

**Your answer here**  
All preplexity scores are lower than the value in test case. This is not surprising since we are trarning the model by these training set which means we are gonna predict these value more accurate than test case since we trained by it. 